In [1]:
import cv2
from ultralytics import YOLO
from collections import defaultdict

In [2]:
model=YOLO('yolov9e.pt')

In [3]:
cap = cv2.VideoCapture("d.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fNUMS = cap.get(cv2.CAP_PROP_FRAME_COUNT)

In [4]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("counting_1.mp4", fourcc, fps, size)

In [5]:
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    #得到目标矩形框的左上角和右下角坐标
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    #绘制矩形框
    cv2.rectangle(image, p1, p2, color, thickness=1, lineType=cv2.LINE_AA)
    if label:
        #得到要书写的文本的宽和长，用于给文本绘制背景色
        w, h = cv2.getTextSize(label, 0, fontScale=2 / 3, thickness=1)[0]  
        #确保显示的文本不会超出图片范围
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)     #填充颜色
        #书写文本
        cv2.putText(image,
                label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                0,
                2 / 3,
                txt_color,
                thickness=1,
                lineType=cv2.LINE_AA)

In [7]:
# track_history用于保存目标ID，以及它在各帧的目标位置坐标，这些坐标是按先后顺序存储的
track_history = defaultdict(lambda: [])
#车辆的计数变量
vehicle_in = 0
vehicle_out = 0
 
#视频帧循环
while cap.isOpened():
    #读取一帧图像
    success, frame = cap.read()
 
    if success:
        #在帧上运行YOLOv8跟踪，persist为True表示保留跟踪信息，conf为0.3表示只检测置信值大于0.3的目标
        results = model.track(frame,conf=0.3, persist=True)
        #得到该帧的各个目标的ID
        track_ids = results[0].boxes.id.int().cpu().tolist()
        #遍历该帧的所有目标
        for track_id, box in zip(track_ids, results[0].boxes.data):
            if box[-1] == 2:   #目标为小汽车
                #绘制该目标的矩形框
                box_label(frame, box, '#'+str(track_id)+' car', (167, 146, 11))
                #得到该目标矩形框的中心点坐标(x, y)
                x1, y1, x2, y2 = box[:4]
                x = (x1+x2)/2
                y = (y1+y2)/2
                #提取出该ID的以前所有帧的目标坐标，当该ID是第一次出现时，则创建该ID的字典
                track = track_history[track_id]
                track.append((float(x), float(y)))  #追加当前目标ID的坐标
                #只有当track中包括两帧以上的情况时，才能够比较前后坐标的先后位置关系
                if len(track) > 1:
                    _, h = track[-2]  #提取前一帧的目标纵坐标
                    #我们设基准线为纵坐标是size[1]-400的水平线
                    #当前一帧在基准线的上面，当前帧在基准线的下面时，说明该车是从上往下运行
                    if h < size[1]-400 and y >= size[1]-400:
                        vehicle_out +=1      #out计数加1
                    #当前一帧在基准线的下面，当前帧在基准线的上面时，说明该车是从下往上运行
                    if h > size[1]-400 and y <= size[1]-400:
                        vehicle_in +=1       #in计数加1
                      
            elif box[-1] == 5:   #目标为巴士
                box_label(frame, box, '#'+str(track_id)+' bus', (67, 161, 255))
                
                x1, y1, x2, y2 = box[:4]
                x = (x1+x2)/2
                y = (y1+y2)/2
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 1:
                    _, h = track[-2]
                    if h < size[1]-400 and y >= size[1]-400:
                        vehicle_out +=1
                    if h > size[1]-400 and y <= size[1]-400:
                        vehicle_in +=1 
               
            elif box[-1] == 7:   #目标为卡车
                box_label(frame, box, '#'+str(track_id)+' truck', (19, 222, 24))
                
                x1, y1, x2, y2 = box[:4]
                x = (x1+x2)/2
                y = (y1+y2)/2
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 1:
                    _, h = track[-2]
                    if h < size[1]-400 and y >= size[1]-400:
                        vehicle_out +=1
                    if h > size[1]-400 and y <= size[1]-400:
                        vehicle_in +=1 
              
            elif box[-1] == 3:   #目标为摩托车
                box_label(frame, box,'#'+str(track_id)+' motor', (186, 55, 2))
                
                x1, y1, x2, y2 = box[:4]
                x = (x1+x2)/2
                y = (y1+y2)/2
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 1:
                    _, h = track[-2]
                    if h < size[1]-400 and y >= size[1]-400:
                        vehicle_out +=1
                    if h > size[1]-400 and y <= size[1]-400:
                        vehicle_in +=1 
        #绘制基准线
        cv2.line(frame, (30,size[1]-400), (size[0]-30,size[1]-400), color=(25, 33, 189), thickness=2, lineType=4)
        #实时显示进、出车辆的数量
        cv2.putText(frame, 'in: '+str(vehicle_in), (595, size[1]-410),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(frame, 'out: '+str(vehicle_out), (573, size[1]-370),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
 
        cv2.putText(frame, "a", (25, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
      
        cv2.imshow("YOLOv8 Tracking", frame)  #显示标记好的当前帧图像
        videoWriter.write(frame)  #写入保存
     
        if cv2.waitKey(1) & 0xFF == ord("q"):   #'q'按下时，终止运行
            break
 
    else:  #视频播放结束时退出循环
        break
 
#释放视频捕捉对象，并关闭显示窗口
cap.release()
videoWriter.release()
cv2.destroyAllWindows()


0: 384x640 3 cars, 1 truck, 274.6ms
Speed: 6.0ms preprocess, 274.6ms inference, 2290.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 209.8ms
Speed: 4.0ms preprocess, 209.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 45.0ms
Speed: 3.0ms preprocess, 45.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 44.9ms
Speed: 3.6ms preprocess, 44.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 49.6ms
Speed: 2.0ms preprocess, 49.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 41.9ms
Speed: 3.5ms preprocess, 41.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 42.9ms
Speed: 3.0ms preprocess, 42.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 42.0ms
Speed: 2.0ms preprocess, 4


0: 384x640 4 cars, 1 truck, 42.1ms
Speed: 2.5ms preprocess, 42.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 41.5ms
Speed: 2.0ms preprocess, 41.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 41.8ms
Speed: 3.6ms preprocess, 41.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 42.0ms
Speed: 2.0ms preprocess, 42.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 41.8ms
Speed: 2.5ms preprocess, 41.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 42.2ms
Speed: 2.0ms preprocess, 42.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 42.2ms
Speed: 2.0ms preprocess, 42.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 41.7ms
Speed: 3.0ms preprocess, 41.7ms i


0: 384x640 4 cars, 1 truck, 41.9ms
Speed: 2.0ms preprocess, 41.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 40.2ms
Speed: 5.1ms preprocess, 40.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 37.4ms
Speed: 2.4ms preprocess, 37.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 42.5ms
Speed: 2.0ms preprocess, 42.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 40.1ms
Speed: 0.0ms preprocess, 40.1ms inference, 7.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 43.7ms
Speed: 1.1ms preprocess, 43.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 42.8ms
Speed: 1.0ms preprocess, 42.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 39.5ms
Speed: 2.5ms preprocess, 39.5ms i


0: 384x640 4 cars, 1 truck, 41.5ms
Speed: 2.0ms preprocess, 41.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 45.4ms
Speed: 2.0ms preprocess, 45.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 42.4ms
Speed: 2.0ms preprocess, 42.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 47.2ms
Speed: 2.0ms preprocess, 47.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 42.2ms
Speed: 2.0ms preprocess, 42.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 42.0ms
Speed: 2.0ms preprocess, 42.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 42.0ms
Speed: 2.9ms preprocess, 42.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 43.1ms
Speed: 2.5ms preprocess, 43.1ms i


0: 384x640 3 cars, 1 truck, 42.0ms
Speed: 2.0ms preprocess, 42.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 42.5ms
Speed: 3.0ms preprocess, 42.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 43.0ms
Speed: 2.1ms preprocess, 43.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 43.0ms
Speed: 2.0ms preprocess, 43.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 41.5ms
Speed: 3.0ms preprocess, 41.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 42.8ms
Speed: 3.0ms preprocess, 42.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 43.5ms
Speed: 2.0ms preprocess, 43.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 41.5ms
Speed: 3.0ms preprocess, 41.5ms i


0: 384x640 2 cars, 1 truck, 42.5ms
Speed: 2.0ms preprocess, 42.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.7ms
Speed: 2.5ms preprocess, 42.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 58.7ms
Speed: 2.0ms preprocess, 58.7ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 48.0ms
Speed: 1.5ms preprocess, 48.0ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.3ms
Speed: 2.2ms preprocess, 42.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 43.0ms
Speed: 3.8ms preprocess, 43.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.0ms
Speed: 1.0ms preprocess, 42.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.0ms
Speed: 3.2ms preprocess, 42.0ms i


0: 384x640 2 cars, 1 truck, 56.5ms
Speed: 2.0ms preprocess, 56.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 41.5ms
Speed: 3.0ms preprocess, 41.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 41.1ms
Speed: 3.0ms preprocess, 41.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 43.0ms
Speed: 3.0ms preprocess, 43.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 43.4ms
Speed: 2.0ms preprocess, 43.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.6ms
Speed: 2.0ms preprocess, 42.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.4ms
Speed: 1.5ms preprocess, 42.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.4ms
Speed: 2.5ms preprocess, 42.4ms i


0: 384x640 2 cars, 1 truck, 42.6ms
Speed: 3.0ms preprocess, 42.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.3ms
Speed: 2.0ms preprocess, 42.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 46.0ms
Speed: 3.0ms preprocess, 46.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.0ms
Speed: 3.0ms preprocess, 42.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.5ms
Speed: 2.3ms preprocess, 42.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.3ms
Speed: 2.5ms preprocess, 42.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.0ms
Speed: 3.0ms preprocess, 42.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 43.1ms
Speed: 3.0ms preprocess, 43.1ms i


0: 384x640 2 cars, 1 truck, 42.2ms
Speed: 3.0ms preprocess, 42.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.0ms
Speed: 3.0ms preprocess, 42.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 53.6ms
Speed: 2.0ms preprocess, 53.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.7ms
Speed: 3.0ms preprocess, 42.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.5ms
Speed: 2.0ms preprocess, 42.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 42.8ms
Speed: 1.0ms preprocess, 42.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 44.0ms
Speed: 2.0ms preprocess, 44.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 42.0ms
Speed: 2.0ms preprocess, 42.0ms inference,